<a href="https://colab.research.google.com/github/mpmailwane/test_repo_/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installations and Library set up**

In [1]:
!apt-get update

# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# download spark3.1
!wget -q http://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [2]:
# bringing data from spark DF to pandas DF: >> If you want to use packages like matplotlib or some of the python packages.
!pip install -U pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 35.4 MB 321 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!ls -a

.   .config	 spark-3.1.2-bin-hadoop2.7
..  sample_data  spark-3.1.2-bin-hadoop2.7.tgz


In [2]:
import findspark
findspark.find()
findspark.init()

##### import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, count, isnan, desc, round

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler

from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import MultiClassificationEvaluator

from operator import add

#from dataprep.eda import create_report
import matplotlib.pyplot as plt
import seaborn as sns

import gdown

ModuleNotFoundError: ignored

In [5]:
spark = SparkSession.builder.master("local[*]").appName("Test Spark").getOrCreate()

In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=Test Spark>

In [8]:
spark

# **Exploratory Data Analysis**

In [9]:
url_2019_oct = "https://drive.google.com/file/d/11lFSrrEh9fPMSPFb3zzKsAZnzKn0KQtG/view?usp=sharing"

oct_2019 = "2019-oct.csv"

In [10]:
gdown.download(url=url_2019_oct, output=oct_2019, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=11lFSrrEh9fPMSPFb3zzKsAZnzKn0KQtG
To: /content/2019-oct.csv
100%|██████████| 5.67G/5.67G [00:45<00:00, 123MB/s]


'2019-oct.csv'

In [11]:
df_2019_oct = spark.read.format("csv").load(oct_2019, header=True, inferSchema=True)

In [12]:
df_2019_oct.show(n=5, truncate=False, vertical=False)

+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|null                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01 UTC|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |null    |543.1  |519107250|566511c2-e2e3-422b-b695-cf8e6e

In [13]:
df_2019_oct.groupBy('event_type').count().show()

+----------+--------+
|event_type|   count|
+----------+--------+
|  purchase|  742849|
|      view|40779399|
|      cart|  926516|
+----------+--------+



# **Feature Engineering**

In [14]:
df_2019_oct_ch = df_2019_oct.dropDuplicates()

In [15]:
df_2019_oct_ch = df_2019_oct_ch.na.drop(how="any")

In [16]:
df_2019_oct_ch.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_2019_oct_ch.columns)).show()

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|         0|         0|         0|          0|            0|    0|    0|      0|           0|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



In [17]:
# After missing values are dropped
(df_2019_oct_ch.count(), len(df_2019_oct_ch.columns))

(26534109, 9)

# **Target Analysis**

In [18]:
# Event Type Is Purchase

target_analysis_purchase = df_2019_oct_ch.select('*').where(df_2019_oct_ch.event_type == 'purchase')

target_analysis_purchase.show(5, truncate = False)

+-----------------------+----------+----------+-------------------+----------------------+-----+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code         |brand|price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+----------------------+-----+-------+---------+------------------------------------+
|2019-10-01 04:12:09 UTC|purchase  |1801551   |2053013554415534427|electronics.video.tv  |haier|463.31 |530014411|6100ba3e-30a5-4574-b875-0b7469688b02|
|2019-10-01 04:28:25 UTC|purchase  |5100337   |2053013553341792533|electronics.clocks    |apple|311.84 |516278990|acefd607-1a24-496c-a4b8-7e9097a71ddf|
|2019-10-01 04:55:41 UTC|purchase  |1005105   |2053013555631882655|electronics.smartphone|apple|1415.48|522740768|dbb10484-ed90-4532-a344-bb05e08c9252|
|2019-10-01 04:58:17 UTC|purchase  |1004250   |2053013555631882655|electronics.smartphon

In [19]:
target_analysis_purchase = target_analysis_purchase.select(['event_type', 'event_time','product_id', 'category_id', 'category_code', 'brand', 'price'])
target_analysis_purchase.show(5, truncate = False)

+----------+-----------------------+----------+-------------------+----------------------+-----+-------+
|event_type|event_time             |product_id|category_id        |category_code         |brand|price  |
+----------+-----------------------+----------+-------------------+----------------------+-----+-------+
|purchase  |2019-10-01 04:12:09 UTC|1801551   |2053013554415534427|electronics.video.tv  |haier|463.31 |
|purchase  |2019-10-01 04:28:25 UTC|5100337   |2053013553341792533|electronics.clocks    |apple|311.84 |
|purchase  |2019-10-01 04:55:41 UTC|1005105   |2053013555631882655|electronics.smartphone|apple|1415.48|
|purchase  |2019-10-01 04:58:17 UTC|1004250   |2053013555631882655|electronics.smartphone|apple|804.21 |
|purchase  |2019-10-01 05:23:30 UTC|1004258   |2053013555631882655|electronics.smartphone|apple|733.92 |
+----------+-----------------------+----------+-------------------+----------------------+-----+-------+
only showing top 5 rows



In [20]:
group1 = target_analysis_purchase.groupby("event_type","product_id","brand").agg(round(sum('price'),2).alias("Sum_price")).orderBy(col('Sum_price').desc())
group1.show(truncate = False)

+----------+----------+-------+-------------+
|event_type|product_id|brand  |Sum_price    |
+----------+----------+-------+-------------+
|purchase  |1005115   |apple  |1.240483595E7|
|purchase  |1005105   |apple  |1.023924868E7|
|purchase  |1004249   |apple  |6729380.83   |
|purchase  |1005135   |apple  |5567806.64   |
|purchase  |1004767   |samsung|5430222.72   |
|purchase  |1002544   |apple  |4854785.55   |
|purchase  |1004856   |samsung|3798168.71   |
|purchase  |1002524   |apple  |3538299.12   |
|purchase  |1003317   |apple  |3051294.26   |
|purchase  |1004870   |samsung|3027098.05   |
|purchase  |1003306   |apple  |2552587.18   |
|purchase  |1004237   |apple  |2449741.88   |
|purchase  |1004258   |apple  |2332809.71   |
|purchase  |1004873   |samsung|2269184.63   |
|purchase  |1004833   |samsung|2184501.96   |
|purchase  |1005106   |apple  |2000494.05   |
|purchase  |4804056   |apple  |1989706.86   |
|purchase  |1005116   |apple  |1975750.52   |
|purchase  |1002633   |apple  |190

In [21]:
group2 = target_analysis_purchase.groupby("event_type","product_id","category_code","brand").agg(count('brand').alias("count_brand")).orderBy(col('count_brand').desc())
group2.show(truncate = False)

+----------+----------+---------------------------+-------+-----------+
|event_type|product_id|category_code              |brand  |count_brand|
+----------+----------+---------------------------+-------+-----------+
|purchase  |1004856   |electronics.smartphone     |samsung|28938      |
|purchase  |1004767   |electronics.smartphone     |samsung|21804      |
|purchase  |1004833   |electronics.smartphone     |samsung|12697      |
|purchase  |1005115   |electronics.smartphone     |apple  |12541      |
|purchase  |4804056   |electronics.audio.headphone|apple  |12379      |
|purchase  |1004870   |electronics.smartphone     |samsung|10615      |
|purchase  |1002544   |electronics.smartphone     |apple  |10547      |
|purchase  |1004249   |electronics.smartphone     |apple  |9089       |
|purchase  |1004836   |electronics.smartphone     |samsung|7689       |
|purchase  |1005105   |electronics.smartphone     |apple  |7293       |
|purchase  |1002524   |electronics.smartphone     |apple  |6633 

In [22]:
group3 = target_analysis_purchase.groupby("event_type","product_id","brand","category_code").agg(count('category_code').alias("count_category_code")).orderBy(col('count_category_code').desc())
group3.show(truncate = False)

+----------+----------+-------+---------------------------+-------------------+
|event_type|product_id|brand  |category_code              |count_category_code|
+----------+----------+-------+---------------------------+-------------------+
|purchase  |1004856   |samsung|electronics.smartphone     |28938              |
|purchase  |1004767   |samsung|electronics.smartphone     |21804              |
|purchase  |1004833   |samsung|electronics.smartphone     |12697              |
|purchase  |1005115   |apple  |electronics.smartphone     |12541              |
|purchase  |4804056   |apple  |electronics.audio.headphone|12379              |
|purchase  |1004870   |samsung|electronics.smartphone     |10615              |
|purchase  |1002544   |apple  |electronics.smartphone     |10547              |
|purchase  |1004249   |apple  |electronics.smartphone     |9089               |
|purchase  |1004836   |samsung|electronics.smartphone     |7689               |
|purchase  |1005105   |apple  |electroni

In [48]:
group4 = target_analysis_purchase.groupby("event_type","brand").agg(count('brand').alias("count_brand2")).orderBy(col('count_brand2').desc())
group4.show(truncate = False)

+----------+--------+------------+
|event_type|brand   |count_brand2|
+----------+--------+------------+
|purchase  |samsung |171688      |
|purchase  |apple   |142562      |
|purchase  |xiaomi  |46588       |
|purchase  |huawei  |23292       |
|purchase  |oppo    |10887       |
|purchase  |lg      |7829        |
|purchase  |acer    |6880        |
|purchase  |elenberg|5434        |
|purchase  |indesit |5023        |
|purchase  |artel   |4717        |
|purchase  |lenovo  |4578        |
|purchase  |bosch   |4055        |
|purchase  |beko    |3837        |
|purchase  |hp      |3411        |
|purchase  |haier   |3043        |
|purchase  |casio   |2660        |
|purchase  |tefal   |2598        |
|purchase  |midea   |2326        |
|purchase  |vitek   |2292        |
|purchase  |sony    |2233        |
+----------+--------+------------+
only showing top 20 rows



In [49]:
group5 = target_analysis_purchase.groupby("event_type","product_id").agg(count('product_id').alias("count_product_id")).orderBy(col('count_product_id').desc())
group5.show(truncate = False)

+----------+----------+----------------+
|event_type|product_id|count_product_id|
+----------+----------+----------------+
|purchase  |1004856   |28938           |
|purchase  |1004767   |21804           |
|purchase  |1004833   |12697           |
|purchase  |1005115   |12541           |
|purchase  |4804056   |12379           |
|purchase  |1004870   |10615           |
|purchase  |1002544   |10547           |
|purchase  |1004249   |9089            |
|purchase  |1004836   |7689            |
|purchase  |1005105   |7293            |
|purchase  |1002524   |6633            |
|purchase  |1004873   |6053            |
|purchase  |1002633   |5301            |
|purchase  |1004741   |5275            |
|purchase  |1005100   |4637            |
|purchase  |1004750   |4597            |
|purchase  |1003306   |4375            |
|purchase  |1004739   |4243            |
|purchase  |1004838   |3976            |
|purchase  |1004565   |3612            |
+----------+----------+----------------+
only showing top

In [23]:
# save file for MapReduce purpose 
target_analysis_purchase_pandas = target_analysis_purchase.select("*").toPandas()

target_analysis_purchase_pandas.to_csv('df_mapReduce.txt', index=None)

### **RDD implementation**

In [24]:
target_analysis_purchase.show(5)

+----------+--------------------+----------+-------------------+--------------------+-----+-------+
|event_type|          event_time|product_id|        category_id|       category_code|brand|  price|
+----------+--------------------+----------+-------------------+--------------------+-----+-------+
|  purchase|2019-10-01 04:12:...|   1801551|2053013554415534427|electronics.video.tv|haier| 463.31|
|  purchase|2019-10-01 04:28:...|   5100337|2053013553341792533|  electronics.clocks|apple| 311.84|
|  purchase|2019-10-01 04:55:...|   1005105|2053013555631882655|electronics.smart...|apple|1415.48|
|  purchase|2019-10-01 04:58:...|   1004250|2053013555631882655|electronics.smart...|apple| 804.21|
|  purchase|2019-10-01 05:23:...|   1004258|2053013555631882655|electronics.smart...|apple| 733.92|
+----------+--------------------+----------+-------------------+--------------------+-----+-------+
only showing top 5 rows



In [25]:
# Converting dataframe into RDD
myRdd = target_analysis_purchase.rdd

In [26]:
rdds = myRdd.map(lambda loop: ((
     loop["event_type"], loop["product_id"], loop["category_code"], loop["brand"])), 1
  )

In [27]:
rdds_brand = myRdd.map(lambda loop: ((loop["brand"]), 1)
  )

In [28]:
rdds_product = myRdd.map(lambda loop: ((loop["product_id"]), 1)
  )

In [29]:
# MapReduce on Brands
rdds_brand.reduceByKey(add).sortBy(lambda x: [-x[1], x[0]]).collect()

[('samsung', 171688),
 ('apple', 142562),
 ('xiaomi', 46588),
 ('huawei', 23292),
 ('oppo', 10887),
 ('lg', 7829),
 ('acer', 6880),
 ('elenberg', 5434),
 ('indesit', 5023),
 ('artel', 4717),
 ('lenovo', 4578),
 ('bosch', 4055),
 ('beko', 3837),
 ('hp', 3411),
 ('haier', 3043),
 ('casio', 2660),
 ('tefal', 2598),
 ('midea', 2326),
 ('vitek', 2292),
 ('sony', 2233),
 ('dauscher', 2223),
 ('nokia', 2218),
 ('redmond', 2114),
 ('philips', 2045),
 ('vivo', 2025),
 ('starline', 1954),
 ('jbl', 1912),
 ('asus', 1826),
 ('pioneer', 1796),
 ('meizu', 1730),
 ('respect', 1728),
 ('kivi', 1584),
 ('ariston', 1502),
 ('polaris', 1488),
 ('epson', 1291),
 ('prestigio', 1210),
 ('janome', 1196),
 ('braun', 1069),
 ('elari', 1020),
 ('huter', 989),
 ('scarlett', 989),
 ('atlant', 931),
 ('arg', 930),
 ('zeta', 851),
 ('kingston', 812),
 ('panasonic', 768),
 ('baden', 757),
 ('oasis', 741),
 ('tcl', 731),
 ('sho-me', 707),
 ('karcher', 662),
 ('asel', 608),
 ('sv', 602),
 ('belecoo', 596),
 ('hansa', 

In [30]:
# MapReduce on Products
rdds_product.reduceByKey(add).sortBy(lambda x: [-x[1], x[0]]).collect()

[(1004856, 28938),
 (1004767, 21804),
 (1004833, 12697),
 (1005115, 12541),
 (4804056, 12379),
 (1004870, 10615),
 (1002544, 10547),
 (1004249, 9089),
 (1004836, 7689),
 (1005105, 7293),
 (1002524, 6633),
 (1004873, 6053),
 (1002633, 5301),
 (1004741, 5275),
 (1005100, 4637),
 (1004750, 4597),
 (1003306, 4375),
 (1004739, 4243),
 (1004838, 3976),
 (1004565, 3612),
 (4804295, 3567),
 (1004858, 3534),
 (1004785, 3506),
 (4804055, 3410),
 (1003317, 3220),
 (1005135, 3218),
 (1004258, 3152),
 (1004209, 2882),
 (1801690, 2744),
 (1004768, 2607),
 (1004659, 2582),
 (1005160, 2433),
 (1004857, 2299),
 (1004237, 2249),
 (1004246, 2248),
 (1004777, 1947),
 (1005116, 1862),
 (1004781, 1849),
 (1005008, 1813),
 (1003316, 1811),
 (1004834, 1772),
 (1003304, 1758),
 (1004250, 1702),
 (1003310, 1690),
 (1005098, 1685),
 (1004903, 1684),
 (3700926, 1675),
 (1005104, 1631),
 (1004766, 1594),
 (1801766, 1561),
 (1004863, 1546),
 (1004708, 1529),
 (1004839, 1523),
 (1004227, 1501),
 (1002547, 1499),
 (3

In [31]:
rdds.getNumPartitions()

200

# **Modelling**

### **Event_type** as our **Target Variable**: we predict which product will be purchased the most so we know what and how much to order.

In [32]:
df_2019_oct_ch.show(5, truncate = False)

+-----------------------+----------+----------+-------------------+--------------------------------+-------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                   |brand  |price |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+--------------------------------+-------+------+---------+------------------------------------+
|2019-10-01 00:02:55 UTC|view      |1005115   |2053013555631882655|electronics.smartphone          |apple  |975.57|516461845|545c5b25-a2db-4b48-84e4-7e44e7b0cf82|
|2019-10-01 02:19:02 UTC|view      |2800511   |2053013563835941749|appliances.kitchen.refrigerators|hansa  |257.38|554016436|1e3e7e1d-5f10-430d-95d8-cf04331d29bc|
|2019-10-01 02:19:16 UTC|view      |21408246  |2053013561579406073|electronics.clocks              |tissot |586.89|543091275|0a7c63ce-37cb-4646-9f95-69f485e391f3|
|2019-10-01 02:19:20 U

In [33]:
# select relevant columns according to the Use Case
brand_df = df_2019_oct_ch.select(["event_type","product_id","brand","price"])   # "category_id",

In [34]:
brand_df.show(5, truncate = False)

+----------+----------+-------+------+
|event_type|product_id|brand  |price |
+----------+----------+-------+------+
|view      |1005115   |apple  |975.57|
|view      |2800511   |hansa  |257.38|
|view      |21408246  |tissot |586.89|
|view      |5100564   |samsung|259.98|
|view      |28708392  |strobbs|31.92 |
+----------+----------+-------+------+
only showing top 5 rows



In [35]:
indexer = StringIndexer(inputCol="brand",
                        outputCol="brand_index")

indexer = indexer.fit(brand_df)
brand_df = indexer.transform(brand_df)

In [36]:
indexer2 = StringIndexer(inputCol="event_type",
                        outputCol="event_type_index")

indexer2 = indexer2.fit(brand_df)
brand_df = indexer2.transform(brand_df)

In [37]:
featureassembler = VectorAssembler(inputCols=['product_id','brand_index','price'],
                          outputCol='independent_features_vector')

brand_df = featureassembler.transform(brand_df)

In [38]:
brand_df.show(5, truncate = False)

+----------+----------+-------+------+-----------+----------------+---------------------------+
|event_type|product_id|brand  |price |brand_index|event_type_index|independent_features_vector|
+----------+----------+-------+------+-----------+----------------+---------------------------+
|view      |1005115   |apple  |975.57|1.0        |0.0             |[1005115.0,1.0,975.57]     |
|view      |2800511   |hansa  |257.38|51.0       |0.0             |[2800511.0,51.0,257.38]    |
|view      |21408246  |tissot |586.89|91.0       |0.0             |[2.1408246E7,91.0,586.89]  |
|view      |5100564   |samsung|259.98|0.0        |0.0             |[5100564.0,0.0,259.98]     |
|view      |28708392  |strobbs|31.92 |81.0       |0.0             |[2.8708392E7,81.0,31.92]   |
+----------+----------+-------+------+-----------+----------------+---------------------------+
only showing top 5 rows



In [39]:
finalized_df = brand_df.select(['independent_features_vector', 'event_type_index'])

In [40]:
finalized_df.show(5, truncate = False)

+---------------------------+----------------+
|independent_features_vector|event_type_index|
+---------------------------+----------------+
|[1005115.0,1.0,975.57]     |0.0             |
|[2800511.0,51.0,257.38]    |0.0             |
|[2.1408246E7,91.0,586.89]  |0.0             |
|[5100564.0,0.0,259.98]     |0.0             |
|[2.8708392E7,81.0,31.92]   |0.0             |
+---------------------------+----------------+
only showing top 5 rows



In [41]:
# split dataset to training and test components
train, test = finalized_df.randomSplit([.7,.3], seed=42)

In [42]:
[train.count(), test.count()]

[18573815, 7960294]

In [43]:
train.show(5, truncate = False)

+---------------------------+----------------+
|independent_features_vector|event_type_index|
+---------------------------+----------------+
|[1000978.0,0.0,301.14]     |0.0             |
|[1000978.0,0.0,301.14]     |0.0             |
|[1000978.0,0.0,301.14]     |0.0             |
|[1000978.0,0.0,333.49]     |0.0             |
|[1000978.0,0.0,333.49]     |0.0             |
+---------------------------+----------------+
only showing top 5 rows



### **Logistic Regression**

In [44]:
log_reg = LogisticRegression(labelCol='event_type_index', featuresCol='independent_features_vector')
model_1 = log_reg.fit(train)

In [45]:
summary = model_1.summary
summary.predictions.describe().show(truncate = False)

+-------+-------------------+----------+
|summary|event_type_index   |prediction|
+-------+-------------------+----------+
|count  |18573815           |18573815  |
|mean   |0.0710442631198814 |0.0       |
|stddev |0.32779758191394764|0.0       |
|min    |0.0                |0.0       |
|max    |2.0                |0.0       |
+-------+-------------------+----------+



In [46]:
log_pred = model_1.evaluate(test)
log_pred.predictions.show()

+---------------------------+----------------+--------------------+--------------------+----------+
|independent_features_vector|event_type_index|       rawPrediction|         probability|prediction|
+---------------------------+----------------+--------------------+--------------------+----------+
|       [1000978.0,0.0,30...|             0.0|[2.19637420369892...|[0.92853514971185...|       0.0|
|       [1000978.0,0.0,33...|             0.0|[2.20283466444879...|[0.92924824167980...|       0.0|
|       [1000978.0,0.0,33...|             0.0|[2.20283466444879...|[0.92924824167980...|       0.0|
|       [1000978.0,0.0,33...|             0.0|[2.20283466444879...|[0.92924824167980...|       0.0|
|       [1000978.0,0.0,33...|             0.0|[2.20283466444879...|[0.92924824167980...|       0.0|
|       [1001588.0,23.0,1...|             0.0|[2.25794478000428...|[0.93520987424174...|       0.0|
|       [1001588.0,23.0,1...|             0.0|[2.25795875936293...|[0.93521126346592...|       0.0|


In [ ]:
evaluator = MultiClassificationEvaluator(rawPredictionCol='prediction',labelCol='event_type_index')

In [ ]:
evaluator.evaluate(log_pred.predictions)